## Interoperabilidade entre Dataframe e RDD
Neste notebook veremos como utilizar os comandos .map e .flatMap para transitar entre RDD e Dataframe. Afinal, atrás de todo Dataframe existe um grande RDD :)<br>
Adicionalmente, aprenderemos a utilizar lambda functions!

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, lit, concat, upper

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
# Inicia uma sparkSession
spark = SparkSession \
    .builder \
    .appName("Meu curso de pyspark") \
    .getOrCreate()

### Carregando dados a partir de um arquivo CSV
Nesta etapa estamos carregando os dados que utilizaremos neste notebook

In [7]:
dados = spark \
    .read.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .format("csv") \
    .load("/FileStore/tables/bank_additional_full-3fd09.csv")

### Função map
Spark Dataframes não possui a função MAP diretamente. Na realidade, esta é uma função do RDD associado ao seu Dataframe.<br>
É por este motivo que, para acionar a função .map, devemos antes referenciar o RDD associado ao Dataframe usando o comando .rdd

**Exemplo de uso da funcao map**<br>
Dica: quando estiver pensando na lógica de escrever este tipo de comando, troque .toDF(['Job']).show() por .take(10) para analisar os resultados!

In [10]:
dados.rdd.map(lambda row: Row('job: ' + row.job)).toDF(['Job']).show()

+----------------+
 Job|
+----------------+
 job: housemaid|
 job: services|
 job: services|
 job: admin.|
 job: services|
 job: services|
 job: admin.|
job: blue-collar|
 job: technician|
 job: services|
job: blue-collar|
 job: services|
job: blue-collar|
 job: housemaid|
job: blue-collar|
 job: retired|
job: blue-collar|
job: blue-collar|
job: blue-collar|
 job: management|
+----------------+
only showing top 20 rows

**Outra forma de conseguir o mesmo resultado usando as abordagens que aprendemos até agora**

In [12]:
dados.select("job").withColumn("teste", concat(lit("Job: "), col("job"))).show(5)

+---------+--------------+
 job| teste|
+---------+--------------+
housemaid|Job: housemaid|
 services| Job: services|
 services| Job: services|
 admin.| Job: admin.|
 services| Job: services|
+---------+--------------+
only showing top 5 rows

**Usando a funcao map para aplicar a funcao upper case**

In [14]:
# Veja que apenas aplicando a função MAP, teremos de volta uma lista de valores
res = dados.rdd.map (lambda linha : linha.poutcome.upper()).take(10)
print(res)
print(type(res))

# Não queremos retornar uma lista de string, mas sim um novo Dataframe com a coluna convertida. É por isso que temos que transformar esta "lista de strings" em uma "lista de Rows"
dados.rdd.map (lambda linha : Row(linha.poutcome.upper())).take(10)

['NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT', 'NONEXISTENT']
<class 'list'>
Out[75]: [<Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>,
 <Row('NONEXISTENT')>]

In [15]:
# Fazendo o comando completo...
dados.rdd.map (lambda linha : Row(linha.poutcome.upper())).toDF(['POUT']).show(5)

+-----------+
 POUT|
+-----------+
NONEXISTENT|
NONEXISTENT|
NONEXISTENT|
NONEXISTENT|
NONEXISTENT|
+-----------+
only showing top 5 rows

**Outra forma de conseguir o mesmo resultado usando as abordagens que aprendemos até agora**

In [17]:
dados.select(upper("poutcome")).show(5)

+---------------+
upper(poutcome)|
+---------------+
 NONEXISTENT|
 NONEXISTENT|
 NONEXISTENT|
 NONEXISTENT|
 NONEXISTENT|
+---------------+
only showing top 5 rows

### Função flatMap
flatMap é uma função muito parecida com .map. A diferença é que os resultados são retornados em um único elemento (flat)<br>
Nos exemplos a seguir retornaremos uma lista com a idade das 10 primeiras pessoas do Dataframe

In [19]:
# Usando .map
dados.rdd.map (lambda row : [row.age]).take(10)

Out[50]: [[56], [57], [37], [40], [56], [45], [59], [41], [24], [25]]

In [20]:
# usando .flatMap
dados.rdd.flatMap (lambda row : [row.age]).take(10)

Out[51]: [56, 57, 37, 40, 56, 45, 59, 41, 24, 25]

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/